# 이중차분법
이중차분법(Difference-in-Differences, DID) 분석 방법을 사용하여 실험 데이터를 분석하는 과정을 단계별로 진행합니다. DID 방법은 정책 변화나 프로그램 도입 등의 효과를 평가하는 데 사용됩니다. 여기서는 두 개의 공장(Factory A와 Factory B)의 생산성 변화를 분석합니다.

## 1. 데이터 준비
먼저, 데이터를 로드하고 날짜 형식을 변환합니다.

여기서는 2023년 6월 30일에 특정 조치(예: 새로운 기계 도입 또는 공정 개선)가 시행되었다고 가정합니다. 이 조치가 두 공장의 생산성에 어떤 영향을 미쳤는지를 분석하는 것이 목적입니다.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv("time-series.csv")
data['date'] = pd.to_datetime(data['date'])

# 분석을 위한 처리 날짜 설정
treatment_date = pd.to_datetime('2023-06-30')

## 2. 데이터 시각화
데이터를 시각화하여 시간에 따른 두 공장의 생산성 변화를 확인합니다. 이 그래프를 통해 두 공장의 생산성이 처리 날짜 전후로 어떻게 변화했는지 시각적으로 확인할 수 있습니다.

In [ ]:
import matplotlib.pyplot as plt

# 시각화
plt.figure(figsize=(12, 6))
for factory in ['Factory A', 'Factory B']:
    factory_data = data[data['factory'] == factory]
    plt.plot(factory_data['date'], factory_data['productivity'], label=factory)

# 처리 날짜를 그래프에 표시
plt.axvline(treatment_date, color='r', linestyle='--', label='Treatment Date')
plt.legend()
plt.title('Factory Productivity Over Time')
plt.xlabel('Date')
plt.ylabel('Productivity')
plt.show()

## 3. 데이터 준비: DID 분석을 위한 변수 추가
DID 분석을 위해 데이터에 추가 변수를 생성합니다. 여기서 Post 변수는 처리 날짜 이후를 나타내며, Treatment 변수는 처리(프로그램 도입)를 받은 그룹(Factory A)을 나타냅니다. Interaction 변수는 Post와 Treatment의 상호작용(term)을 나타냅니다.

In [ ]:
# 데이터에 처치 변수 추가
data['post'] = (data['date'] >= treatment_date).astype(int)
data['treatment'] = (data['factory'] == 'Factory A').astype(int)
data['interaction'] = data['post'] * data['treatment']

이중차분법 (Difference-in-Differences) 회귀 분석을 통해 조치의 효과를 추정해보겠습니다. 이 분석 방법은 두 시점 간의 변화와 두 그룹 간의 차이를 비교하여 조치의 실제 효과를 평가합니다.

**왜 이런 식으로 회귀 모델을 사용할까요?**

우리가 관심 있는 것은 특정 조치(예: 새로운 기계 도입)가 공장의 생산성에 어떤 영향을 미쳤는지를 파악하는 것입니다. 이를 위해 DID 분석은 조치 전후의 변화와 조치를 받은 그룹과 받지 않은 그룹 간의 차이를 함께 고려합니다.

회귀 모델에서 사용되는 변수들은 다음과 같습니다:
- **Post**: 조치가 시행된 후(1)인지 이전(0)인지를 나타냅니다.
- **Treatment**: 조치를 받은 그룹(Factory A, 1)인지 받지 않은 그룹(Factory B, 0)인지를 나타냅니다.
- **Interaction**: 조치 이후와 조치 받은 그룹의 상호작용을 나타냅니다. 이 변수는 조치의 효과를 평가하는 데 사용됩니다.

회귀 모델은 다음과 같이 표현됩니다:

$$
\text{생산성} = \beta_0 + \beta_1 \cdot \text{Post} + \beta_2 \cdot \text{Treatment} + \beta_3 \cdot \text{Interaction} + \epsilon
$$

- **Post**: 이 변수는 조치 이후에 해당하는 경우 1, 조치 이전에 해당하는 경우 0입니다. 이를 통해 시간이 지남에 따른 변화(전후 효과)를 평가할 수 있습니다.
- **Treatment**: 이 변수는 조치를 받은 그룹(Factory A)인 경우 1, 받지 않은 그룹(Factory B)인 경우 0입니다. 이를 통해 그룹 간의 기본 차이를 평가할 수 있습니다.
- **Interaction**: 이 변수는 Post와 Treatment의 곱으로, 조치 이후에 조치를 받은 그룹에 해당하는 경우 1, 그렇지 않은 경우 0입니다. 이 변수는 조치의 실제 효과를 평가하는 데 사용됩니다.

이렇게 설정된 회귀 모델을 통해 우리는 조치가 생산성에 미친 영향을 추정할 수 있습니다. DID 분석은 이러한 설정을 통해 시간의 흐름과 그룹 간의 차이를 함께 고려하여 조치의 순수한 효과를 분리해냅니다. 이는 단순히 조치 전후의 변화나 그룹 간의 차이만을 보는 것보다 더 정확한 평가를 가능하게 합니다.

In [ ]:
from statsmodels.formula.api import ols

# DID 회귀 분석
model = ols('productivity ~ post + treatment + interaction', data=data).fit()
summary = model.summary()
print(summary)

In [ ]:
# DID 추정치
did_estimate = model.params['interaction']
print(f"DID Estimate: {did_estimate:.2f}")